In [16]:
import xml.etree.ElementTree as ET
import pandas as pd
import zipfile
import re
import json
import dateparser
import collections
from lxml import etree
from xml.etree.ElementTree import XML
from random import shuffle
docxFileName = "quick_section4.docx"
docxZip = zipfile.ZipFile(docxFileName)
documentXML = docxZip.read('word/document.xml')
et = etree.XML(documentXML)
ns = {'w': 'http://schemas.openxmlformats.org/wordprocessingml/2006/main'}

In [17]:
mainstation_xpath = './w:r[1][not(w:rPr/w:sz[@w:val="16"])]/w:t[1]'
first_token_para_xpath = './w:r[1]/w:t[1]'

stations = pd.DataFrame(columns=['station','type','description'])


def is_upper(para):
    if para.xpath(first_token_para_xpath, namespaces=ns):
        if para.xpath(first_token_para_xpath, namespaces=ns)[0].text.isupper():
            return 'station'
        else: 
            return 'new_line'


mainstation = ""
lowerstation = ""
dText = dict()
for i, para in enumerate(et.xpath('//w:p', namespaces=ns)):
    text = para.xpath('./w:r/w:t', namespaces=ns)
    description = " ".join([t.text for t in text])
    if is_upper(para) == 'station':
        mainxpath = para.xpath(mainstation_xpath, namespaces=ns)
        if mainxpath and len(mainxpath[0].text) > 1:
            mainstation = mainxpath[0].text
    description = description.lstrip('\x01').strip()
    if description:
        if mainstation in dText:
            dText[mainstation].append(description)# += "\n" + description
        else:
            description = re.sub('^(' + re.escape(mainstation) + ')', '\1', description).lstrip('\x01').strip()
            description = re.sub(r" +", " ", description).lstrip('\x01').strip()
            if description:
                dText[mainstation] = [description]

In [18]:
dStations = collections.OrderedDict(dText)

In [19]:
stations = []
descriptions = []
for k in dStations:
    stations.append(k)
    descriptions.append(dStations[k])

In [20]:
stationdf = pd.DataFrame(columns=["Station", "Description"])
stationdf["Station"] = stations
stationdf["Description"] = descriptions

In [21]:
stationdf.head()

,Station,Description
0,ABBEY,[A TOWN [NB] op 3 September 1856** as A; TOWN ...
1,ABBEY & WEST DEREHAM,[[GE] op 1 August 1882 (Thetford & Watton Time...
2,ABBEY FOREGATE,[– see SHREWSBURY.]
3,ABBEY HOUSES,[see BEAUCHIEF .]
4,ABBEY MILLS,[pumping station [private line south of West H...


In [22]:
d = pd.read_pickle('quick_processed.pkl').iloc[9306]

In [25]:
type(d["Description"])

list

In [26]:
stationdf.to_pickle('quick_processed.pkl')

In [8]:
stationdf_sample = stationdf.sample(n=100, random_state=42)

for i, row in stationdf_sample.iterrows():
    with open("to_annotate/" + str(i) + ".txt", "w") as fw:
        fw.write("[[[STATION: " + row["Station"] + "]]]")
        for d in row["Description"]:
            fw.write("\n" + d)

In [30]:
d = pd.read_pickle('quick_processed.pkl').iloc[9306]
for k in d["Description"]:
    print(k)

[ N&B] opened 19 November 1873 (G&S) as YNISCEDWYN; renamed 1 May 1893 (hbl 27 April) ; closed 12 September 1932 (RM October).


In [15]:
for d in pd.read_pickle('quick_processed.pkl').iloc[9306]["Description"]:
    for k in d:
        print(k)

[
 
N
&
B
]
 
o
p
e
n
e
d
 
1
9
 
N
o
v
e
m
b
e
r
 
1
8
7
3
 
(
G
&
S
)
 
a
s
 
Y
N
I
S
C
E
D
W
Y
N
;
 
r
e
n
a
m
e
d
 
1
 
M
a
y
 
1
8
9
3
 
(
h
b
l
 
2
7
 
A
p
r
i
l
)
 
;
 
c
l
o
s
e
d
 
1
2
 
S
e
p
t
e
m
b
e
r
 
1
9
3
2
 
(
R
M
 
O
c
t
o
b
e
r
)
.
